In [10]:
# Set path to the parent directory
import sys
sys.path.append('../') # let you import from parent directory

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image


import lightgbm as lgb
import cv2
import os
import numpy as np
from sklearn.metrics import accuracy_score

import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from tqdm import tqdm
import csv

# Import .py files and their functions
from bbox import rot, get_bbox
from utils import write_paths, write_labels, write_rotation_vectors, \
                    write_centroids, write_sizes, write_clouds, write_camera_matrices

In [11]:
# Collect and save labels, centroids, point clouds, and camera matrices data
path = 'trainval' # path to folder containing training snapshots, bbox.bin, cloud.bin, proj.bin

# File Paths
if not os.path.exists('./trainval/trainval_paths.npy'):
    write_paths(path) # trainval_paths.npy

if not os.path.exists('./test/test_paths.npy'):
    write_paths('test') # test_paths.npy

# Labels of Bounding Boxes
if not os.path.exists('./trainval/trainval_labels.npy'):
    write_labels(path) # trainval_labels.npy

# Bounding Box Features - Rotation Vectors
if not os.path.exists('./trainval/trainval_rotation_vectors.npy'):
    write_rotation_vectors(path) # trainval_rotations.npy

# Bounding Box Features - Centroids
if not os.path.exists('./trainval/trainval_centroids.npy'):
    write_centroids(path) # trainval_centroids.npy

# Bounding Box Features - Sizes
if not os.path.exists('./trainval/trainval_sizes.npy'):
    write_sizes(path) # trainval_sizes.npy

# Camera Matrices
if not os.path.exists('./trainval/trainval_camera_matrices.npy'):
    write_camera_matrices(path) # trainval_camera_matrices.npy

In [12]:
# Root directory for training data
train_root = './trainval/'
test_root = './test/'

# Load data
train_images = glob('trainval/*/*_image.jpg') # Grab all the training snapshot image paths
test_images = np.load('test/test_images.npy') # Grab all the test snapshot images
train_paths = np.load(train_root + 'trainval_paths.npy') # each training data's path (for reference)
test_paths = np.load(test_root + 'test_paths.npy') # each test data's path (for reference)
labels = np.load(train_root + 'trainval_labels.npy')
rotation_vectors = np.load(train_root + 'trainval_rotation_vectors.npy')
centroids = np.load(train_root + 'trainval_centroids.npy')
sizes = np.load(train_root + 'trainval_sizes.npy')
camera_matrices = np.load(train_root + 'trainval_camera_matrices.npy', allow_pickle=True)

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# def crop_bbox(image, rotation_vector, centroid, size, camera_matrix):
#     """
#     Convert 3D bounding box to 2D bounding box and crop the object from the image.
#     """

#     proj = camera_matrix

#     R = rot(rotation_vector)
#     t = centroid
#     sz = size

#     vert_3D, edges = get_bbox(-sz / 2, sz / 2)

#     vert_3D = R @ vert_3D + t[:, np.newaxis]

#     vert_2D = proj @ np.vstack([vert_3D, np.ones(vert_3D.shape[1])])
#     vert_2D = vert_2D / vert_2D[2, :]

#     x_array = []
#     y_array = []
#     for e in edges.T:
#         x_array.append(vert_2D[0, e])
#         y_array.append(vert_2D[1, e])
#         # plt.plot(vert_2D[0, e], vert_2D[1, e], 'r-')
#     x_array = np.clip(np.array(x_array), 0, image.shape[1]-1) # clip to prevent bbox out of image
#     y_array = np.clip(np.array(y_array), 0, image.shape[0]-1) # clip to prevent bbox out of image
#     x_array = torch.from_numpy(x_array)
#     y_array = torch.from_numpy(y_array)

#     x_min = torch.min(x_array)
#     x_max = torch.max(x_array)
#     y_min = torch.min(y_array)
#     y_max = torch.max(y_array)

#     if x_min == x_max:
#         x_min -= 1
#         x_max += 1
#     elif y_min == y_max:
#         y_min -= 1
#         y_max += 1

#     object_image = image[int(y_min):int(y_max), int(x_min):int(x_max)]
#     object_image = Image.fromarray(object_image)
#     object_image = object_image.resize((224, 224))
#     # object_image = np.array(object_image)

#     return object_image


In [ ]:
# # Create a custom dataset class
# class VehicleDataset(Dataset):
#     def __init__(self, images, rotation_vectors, centroids, sizes, camera_matrices, labels, paths):
#         self.images = images
#         self.rotation_vectors = rotation_vectors
#         self.centroids = centroids
#         self.sizes = sizes
#         self.camera_matrices = camera_matrices
#         self.labels = labels

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         image = plt.imread(self.images[idx])
#         rotation_vector = self.rotation_vectors[idx]
#         centroid = self.centroids[idx]
#         size = self.sizes[idx]
#         camera_matrix = self.camera_matrices[idx]
#         label = self.labels[idx]

#         cropped_image = crop_bbox(image, rotation_vector, centroid, size, camera_matrix)

#         return cropped_image, label

In [ ]:
# # Example
# idxx = random.randint(0, len(train_images))
# print(idxx)
# zz=crop_bbox(plt.imread(train_images[idxx]), rotation_vectors[idxx], centroids[idxx], sizes[idxx], camera_matrices[idxx])
# plt.imshow(plt.imread(train_images[idxx])) # image
# # plt.plot(xx,yy) # 3D bounding box
# plt.show()

# plt.imshow(zz) # cropped image based on 2D bounding box
# plt.show()

In [ ]:
# # Train + Val dataset
# trainval_dataset = VehicleDataset(train_images, rotation_vectors, centroids, 
#                                   sizes, camera_matrices, labels, train_paths)

# # Train Val Split
# train_size = int(0.8 * len(trainval_dataset))
# val_size = len(trainval_dataset) - train_size
# train_dataset, val_dataset = random_split(trainval_dataset, [train_size, val_size])

# # Extract features from the images using a pre-trained model
# resnet = models.resnet18(pretrained=True)
# resnet = torch.nn.Sequential(*(list(resnet.children())[:-1])) # Remove the last layer to get features
# resnet = resnet.to(device)
# resnet.eval()

# transform_train = T.Compose([
#     T.ToTensor(),
#     T.RandomHorizontalFlip(),
#     T.RandomRotation(20),
#     T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

# def extract_features_train(dataset):
#     features = []
#     labels = []
#     with torch.no_grad():
#         for img, label in tqdm(dataset, desc='Extracting features from training images'):
#             img = transform_train(img)
#             img = img.unsqueeze(0)
#             img = img.to(device)
#             feature = resnet(img).squeeze().cpu().numpy()
#             features.append(feature)
#             labels.append(label)
#     return np.array(features), np.array(labels)


In [ ]:
# num_classes = len(set(labels)) # number of classes

# X_train, y_train = extract_features_train(train_dataset)
# X_val, y_val = extract_features_train(val_dataset)

# # Train a LightGBM model on the extracted features
# train_data = lgb.Dataset(X_train, label=y_train)
# params = {
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_class': num_classes, # Replace with the number of classes
#     'boosting_type': 'gbdt',
#     'learning_rate': 0.01,
#     'feature_fraction': 0.9, # random fraction of features to use
#     'bagging_fraction': 0.8, # random fraction of samples of data for each tree
#     'bagging_freq': 5, # frequency of bagging
#     'verbose': 1,
#     'bagging_seed': 42, # random seed for reproducibility
# }

# # Train the model
# model = lgb.train(params, train_data, num_boost_round=3000, 
#                   valid_sets=[train_data], early_stopping_rounds=10)

# # Make predictions and evaluate the model
# y_pred = model.predict(X_val)
# y_pred = np.argmax(y_pred, axis=1)
# accuracy = accuracy_score(y_val, y_pred)

# print("Accuracy:", accuracy)


In [ ]:
class VehicleFullImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = plt.imread(self.images[idx])
        image = Image.fromarray(image)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Create a new dataset for fine-tuning with full images
fine_tune_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

fine_tune_dataset = VehicleFullImageDataset(train_images, labels, fine_tune_transform)

# Split the new dataset into training and validation subsets
train_size = int(0.8 * len(fine_tune_dataset))
val_size = len(fine_tune_dataset) - train_size

fine_tune_train_dataset, fine_tune_val_dataset = random_split(fine_tune_dataset, [train_size, val_size])

# Create data loaders for the fine-tuning datasets
batch_size = 32

fine_tune_train_loader = DataLoader(fine_tune_train_dataset, batch_size=batch_size, shuffle=True)
fine_tune_val_loader = DataLoader(fine_tune_val_dataset, batch_size=batch_size, shuffle=False)

# Use the pre-trained ResNet model
resnet = models.resnet18(pretrained=True)
for module in resnet.modules():
    if isinstance(module, nn.ReLU):
        module.inplace = nn.LeakyReLU(negative_slope=0.01, inplace=True)

# Replace the last layer with a new fully connected layer for classification
# num_classes = len(set(labels))
# resnet.fc = torch.nn.Linear(resnet.fc.in_features, num_classes)

# Add a few more FC layers to the model
num_classes = len(set(labels))
resnet.fc = nn.Sequential(
    nn.Linear(512, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, num_classes)
)

# Freeze all layers except the last 5 layers
for name, param in resnet.named_parameters():
    if "fc" not in name:
        param.requires_grad = False

# Set a smaller learning rate for fine-tuning
fine_tune_lr = 0.001
optimizer = optim.SGD(resnet.parameters(), lr=fine_tune_lr, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

# Train the model for a few more epochs
fine_tune_epochs = 8
resnet = resnet.to(device)

for epoch in range(1, fine_tune_epochs + 1):
    # Training loop
    resnet.train()
    running_loss = 0.0
    for i, (image_batch, label_batch) in tqdm(enumerate(fine_tune_train_loader, 1), total=len(fine_tune_train_loader)):
        input_batch, label_batch = image_batch.to(device), label_batch.to(device)

        optimizer.zero_grad()
        outputs = resnet(input_batch)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    resnet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for input_batch, label_batch in fine_tune_val_loader:
            input_batch, label_batch = input_batch.to(device), label_batch.to(device)

            outputs = resnet(input_batch)
            _, predicted = torch.max(outputs.data, 1)
            total += label_batch.size(0)
            correct += (predicted == label_batch).sum().item()

    # Print the epoch summary
    print(f"Epoch {epoch}/{fine_tune_epochs}, "
          f"Loss: {running_loss / len(fine_tune_train_loader):.4f}, "
          f"Accuracy: {100 * correct / total:.2f}%")



c:\Users\whdqk\anaconda3\envs\sam\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\whdqk\anaconda3\envs\sam\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 190/190 [09:31<00:00,  3.01s/it]


Epoch 1/8, Loss: 0.9633, Accuracy: 63.56%


100%|██████████| 190/190 [08:51<00:00,  2.80s/it]


Epoch 2/8, Loss: 0.8377, Accuracy: 63.56%


100%|██████████| 190/190 [09:46<00:00,  3.09s/it]


KeyboardInterrupt: 

In [ ]:
class VehicleTestImageDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image

test_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

test_batch_size = 1

test_image_dataset = VehicleTestImageDataset(test_images, test_transform)
test_image_loader = DataLoader(test_image_dataset, batch_size=test_batch_size, shuffle=False)

resnet.eval()
test_predictions = []

with torch.no_grad():
    for inputs in tqdm(test_image_loader):
        inputs = inputs.to(device)

        outputs = resnet(inputs)
        _, predicted = torch.max(outputs.data, 1)

        test_predictions.extend(predicted.cpu().numpy())

with open('test_predictions_modified_resnet.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)

    # Write the header
    csv_writer.writerow(["guid/image", "label"])

    for i, pred in tqdm(enumerate(test_predictions), desc='Saving predictions to file'):
        csv_writer.writerow([test_paths[i], pred])


100%|██████████| 2631/2631 [04:06<00:00, 10.69it/s]
Saving predictions to file: 2631it [00:00, 50730.32it/s]


In [ ]:
# idxx = 414

# xx=object_detection_model(transform_test(test_images[idxx]).unsqueeze(0))
# xx[0]['boxes'].shape[0]
# xx = xx[0]['boxes'][0]
# x_min, y_min, x_max, y_max = map(int, xx)
# plt.imshow(test_images[idxx][y_min:y_max, x_min:x_max])